### The data:

The dataset is collected and maintained by UCI Machine Learning Repository. The target variable is `traffic_volume`. The dataset contains the following and has already been normalized and saved into training and test sets:

`train_scaled.csv`, `test_scaled.csv`
| Column     | Type       | Description              |
|------------|------------|--------------------------|
|`temp`                   |Numeric            |Average temp in kelvin|
|`rain_1h`                |Numeric            |Amount in mm of rain that occurred in the hour|
|`snow_1h`                |Numeric            |Amount in mm of snow that occurred in the hour|
|`clouds_all`             |Numeric            |Percentage of cloud cover|
|`date_time`              |DateTime           |Hour of the data collected in local CST time|
|`holiday_` (11 columns)  |Categorical        |US National holidays plus regional holiday, Minnesota State Fair|
|`weather_main_` (11 columns)|Categorical     |Short textual description of the current weather|
|`weather_description_` (35 columns)|Categorical|Longer textual description of the current weather|
|`traffic_volume`         |Numeric            |Hourly I-94 ATR 301 reported westbound traffic volume|
|`hour_of_day`|Numeric|The hour of the day|
|`day_of_week`|Numeric|The day of the week (0=Monday, Sunday=6)|
|`day_of_month`|Numeric|The day of the month|
|`month`|Numeric|The number of the month|
|`traffic_volume`         |Numeric            |Hourly I-94 ATR 301 reported westbound traffic volume|

1. Import Required Libraries

In [36]:
# Import the relevant libraries
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

2. Load and Prepare the Dataset

In [37]:
# Read the traffic data from the CSV training and test files
train_scaled_df = pd.read_csv('train_scaled.csv')
test_scaled_df = pd.read_csv('test_scaled.csv')

# Convert the DataFrame to NumPy arrays
train_scaled = train_scaled_df.to_numpy()
test_scaled = test_scaled_df.to_numpy()

3. Create Sequences from the Time Series Data

In [38]:
def create_sequence(data,seq_length,target_idx):
    inputs = []
    targets = []
    
    for i in range(0, len(data) - seq_length):
        seq_x = data[i:i+seq_length]
        
        seq_y = data[i+seq_length,target_idx]
        
        inputs.append(seq_x)
        targets.append(seq_y)
    return np.array(inputs), np.array(targets)

4. Generate Sequences for Train and Test Sets

In [39]:
X_train ,y_train = create_sequence(train_scaled,24,0)
X_test ,y_test = create_sequence(test_scaled,24,0)

5. Convert NumPy Arrays to PyTorch Tensors

In [40]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).float()
y_train_tensor = torch.tensor(y_train.astype(np.float32)).float()
X_test_tensor = torch.tensor(X_test.astype(np.float32)).float()
y_test_tensor = torch.tensor(y_test.astype(np.float32)).float()

6. Wrap Tensors in TensorDataset and Load with DataLoader

In [41]:
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

7. Define the LSTM Neural Network Model

In [42]:
class TrafficModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super(TrafficModel, self).__init__()
        
        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, 1)
        
        # Activation function
        self.activation = nn.LeakyReLU()
    
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        out, (h_n, c_n) = self.lstm(x)
        
        # h_n shape: (num_layers, batch_size, hidden_size)
        last_hidden = h_n[-1]  # Take the final hidden state
        
        out = self.fc(last_hidden)
        out = self.activation(out)
        
        return out

8. Initialize the Model, Loss Function, and Optimizer

In [43]:
input_size = train_scaled.shape[1]
traffic_model = TrafficModel(input_size=input_size)

In [44]:
# Loss function for regression
criterion = nn.MSELoss()

# Optimizer (you can try Adam, SGD, etc.)
optimizer = optim.Adam(traffic_model.parameters(), lr=0.001)

9. Train the Model

In [29]:
# Train for 2 epochs
num_epochs = 2

for epoch in range(num_epochs):
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        # Reset gradients
        optimizer.zero_grad()

        # Forward pass
        predictions = traffic_model(X_batch)

        # Compute loss
        loss = criterion(predictions.squeeze(), y_batch)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Track the loss
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Store final training loss
final_training_loss = avg_loss

 10. Set the Model to Evaluation Mode

In [30]:
# Put model in evaluation mode
traffic_model.eval()

TrafficModel(
  (lstm): LSTM(66, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (activation): LeakyReLU(negative_slope=0.01)
)

11. Prepare to Store Predictions and Labels

In [31]:
# Lists to store all predictions and true values
all_preds = []
all_labels = []

12. Disable Gradient Tracking & Run Evaluation Loop

In [32]:
# Turn off gradient computation
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        # Get predictions
        outputs = traffic_model(X_batch)

        # Squeeze to match shape
        preds = outputs.squeeze()

        # Store predictions and true labels
        all_preds.append(preds)
        all_labels.append(y_batch)

13. Concatenate All Batches Into Full Prediction Tensors

In [33]:
# Concatenate all predictions and labels into full tensors
all_preds_tensor = torch.cat(all_preds)
all_labels_tensor = torch.cat(all_labels)

14. Compute Mean Squared Error on Test Set

In [34]:
# Compute Mean Squared Error over the entire test set
test_mse = F.mse_loss(all_preds_tensor, all_labels_tensor)

print(f"Test MSE: {test_mse.item():.4f}")

Test MSE: 0.0002
